In [1]:
import redcap


In [2]:
%run -i -n "credentials.py"

print(dev_url)
print(prod_url)
print(["dev_pid100", "dev_pid278", "prod_pid100", "prod_pid278"])

https://qa-redcap.doh.wa.gov/api/
https://redcap.doh.wa.gov/api/
['dev_pid100', 'dev_pid278', 'prod_pid100', 'prod_pid278']


In [3]:
prod100 = redcap.Project(prod_url, prod_pid100)
prod278 = redcap.Project(prod_url, prod_pid278)

dev100 = redcap.Project(dev_url, dev_pid100)
dev278 = redcap.Project(dev_url, dev_pid278)

In [57]:
prod100_df = prod100.export_records(format_type = "df", raw_or_label = "raw")

C:\Users\AXG5303\Miniconda3\envs\transform22\lib\site-packages\redcap\methods\base.py:137: DtypeWarning: Columns (397) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(buf, **df_kwargs)


In [64]:
prod100_df.dtypes

provider_yn                                float64
inq_org_name                                object
inq_fac_name                                object
inq_fac_street                              object
inq_fac_city                                object
inq_fac_state                               object
inq_fac_zip                                float64
inq_contact_fname                           object
inq_contact_lname                           object
inq_contact_phone                           object
inq_contact_ext                            float64
inq_contact_email                           object
inquiry_enrollment_survey_complete           int64
inq_in_iis                                 float64
inq_iis_notes                               object
inq_valid                                  float64
inq_vm_notes                                object
send_survey                                float64
send_rejectionletter                       float64
inquiry_review_form_complete   

In [ ]:
prod278_df = prod278.export_records(format_type = "df")

In [6]:
prod100_df.shape

(2665, 471)

In [ ]:
prod278_df.shape

In [58]:
p100_df = prod100_df.iloc[0:50,0:21].copy()


In [ ]:
p278_df = prod278_df.iloc[0:50,0:21].copy()

In [ ]:
p100_df["provider_interest"]

In [ ]:
p100_df["provider_interest"] = pd.to_numeric(p100_df["provider_interest"], downcast="integer")

In [60]:
csv = p100_df.to_csv()

In [37]:
p100_json = p100_df.to_json(orient = "index")

In [63]:
dev100.import_records(csv, import_format = "csv")

RequestException: {'error': '"1","provider_interest","1.0","The value is not a valid category for provider_interest"\n"2","provider_interest","1.0","The value is not a valid category for provider_interest"\n"3","provider_interest","1.0","The value is not a valid category for provider_interest"\n"4","provider_interest","1.0","The value is not a valid category for provider_interest"\n"5","provider_interest","1.0","The value is not a valid category for provider_interest"\n"6","provider_interest","1.0","The value is not a valid category for provider_interest"\n"7","provider_interest","1.0","The value is not a valid category for provider_interest"\n"8","provider_interest","1.0","The value is not a valid category for provider_interest"\n"9","provider_interest","1.0","The value is not a valid category for provider_interest"\n"10","provider_interest","1.0","The value is not a valid category for provider_interest"\n"11","provider_interest","1.0","The value is not a valid category for provider_i

In [ ]:
dev100.import_records?

In [ ]:
dev100.import_records?

In [53]:
test = p100_df.iloc[0:2,0:21]

In [54]:
test

,record_id,provider_yn,inq_org_name,inq_fac_name,inq_fac_street,inq_fac_city,inq_fac_state,inq_fac_zip,inq_contact_fname,inq_contact_lname,...,inq_contact_ext,inq_contact_email,inquiry_enrollment_survey_complete,inq_in_iis,inq_iis_notes,inq_valid,inq_vm_notes,send_survey,send_rejectionletter,inquiry_review_form_complete
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [55]:
test.to_json()

'{"record_id":{"0":1,"1":2},"provider_yn":{"0":null,"1":null},"inq_org_name":{"0":null,"1":null},"inq_fac_name":{"0":null,"1":null},"inq_fac_street":{"0":null,"1":null},"inq_fac_city":{"0":null,"1":null},"inq_fac_state":{"0":null,"1":null},"inq_fac_zip":{"0":null,"1":null},"inq_contact_fname":{"0":null,"1":null},"inq_contact_lname":{"0":null,"1":null},"inq_contact_phone":{"0":null,"1":null},"inq_contact_ext":{"0":null,"1":null},"inq_contact_email":{"0":null,"1":null},"inquiry_enrollment_survey_complete":{"0":0,"1":0},"inq_in_iis":{"0":null,"1":null},"inq_iis_notes":{"0":null,"1":null},"inq_valid":{"0":null,"1":null},"inq_vm_notes":{"0":null,"1":null},"send_survey":{"0":null,"1":null},"send_rejectionletter":{"0":null,"1":null},"inquiry_review_form_complete":{"0":0,"1":0}}'

In [ ]:
p100meta = prod100.export_metadata(format_type = "df")

In [ ]:
p100meta["field_type"].value_counts()

In [ ]:
numeric = ["checkbox", "yesno", "dropdown", "radio"]

num_fields = list(p100meta[p100meta["field_type"].isin(numeric)].index)

In [ ]:
num_fields_1 = set(num_fields) - set(['loc_setting', 'loc_pop', 'vm_staff_assigned', 'approved_units', 'type_ord_set'])

In [ ]:
prod100_df[list(num_fields_1)]

In [ ]:
p100meta.sort_index(inplace = True)

In [ ]:
[i for i in p100meta.index]

In [ ]:
[i for i in prod100_df.columns.sort_values()]

In [ ]:
loc_set = ['loc_setting___1',
 'loc_setting___10',
 'loc_setting___11',
 'loc_setting___12',
 'loc_setting___13',
 'loc_setting___14',
 'loc_setting___15',
 'loc_setting___16',
 'loc_setting___17',
 'loc_setting___2',
 'loc_setting___3',
 'loc_setting___4',
 'loc_setting___5',
 'loc_setting___6',
 'loc_setting___7',
 'loc_setting___8',
 'loc_setting___9',]

prod100_df[loc_set].dtypes

In [ ]:
p100meta.sort_index(inplace =True)

In [ ]:
p100meta.index

In [ ]:
[i for i in p100meta.index]

In [ ]:
filt = p100meta["form_name"] == "inquiry_enrollment_survey"
p100meta[filt].shape